In [1]:
import numpy as np
import pandas as pd

import multipoles, read_from_dft

pd.set_option('display.float_format', '{:.3f}'.format)

# Reads the occupation matrix from VASP

In [2]:
with open(f'BMRO_benchmark/OUTCAR', 'r') as file:
    occupation_per_site = read_from_dft.read_densmat_from_vasp(file)

/work/DMFT_misc/multipoles_from_densmat/read_from_dft.py:130: UserWarning: The multipoles here are consistent with the 2021 VASP implementation.There has been a bug fix which might change the signs of some magnetic multipoles.
  warnings.warn('The multipoles here are consistent with the 2021 VASP implementation.'


In [3]:
occupation_per_site.shape

(2, 5, 5, 2, 2)

The data has the shape n_atoms x n_orb x n_orb x n_spin x n_spin. Always: n_spin=2, n_orb=2l+1.

Printing the density matrix for the first Re atom and spin (up/up):

In [4]:
with np.printoptions(precision=3, suppress=True, floatmode='fixed'):
    print(occupation_per_site[0, :, :, 0, 0])

[[ 0.414+0.000j -0.004-0.000j  0.007-0.000j -0.004+0.001j  0.000-0.016j]
 [-0.004+0.000j  0.408+0.000j  0.004+0.001j  0.083-0.036j -0.000+0.092j]
 [ 0.007+0.000j  0.004-0.001j  0.413+0.000j  0.003-0.001j -0.001+0.003j]
 [-0.004-0.001j  0.083+0.036j  0.003+0.001j  0.390+0.000j -0.021+0.081j]
 [ 0.000+0.016j -0.000-0.092j -0.001-0.003j -0.021-0.081j  0.388+0.000j]]


# Runs multipole code

In [5]:
results, l = multipoles.calculate(occupation_per_site, verbose=False)

Angular momentum of density matrix is l = 2
Transforming from cubic to spherical harmonics


In [6]:
results

,species,atom,nu,l1,l2,k,p,r,t,value
0,1,0,0,2,2,0,0,0,0,4.026+0.000j
1,1,1,0,2,2,0,0,0,0,4.026+0.000j
2,1,0,1,2,2,0,0,0,0,0.000+0.000j
3,1,1,1,2,2,0,0,0,0,0.000+0.000j
4,1,0,0,2,2,0,1,1,-1,0.000+0.000j
...,...,...,...,...,...,...,...,...,...,...
395,1,1,1,2,2,4,1,5,1,-0.351+0.342j
396,1,1,1,2,2,4,1,5,2,-0.000+0.000j
397,1,1,1,2,2,4,1,5,3,0.452+0.910j
398,1,1,1,2,2,4,1,5,4,0.001+0.000j


In [7]:
k, p, r = (2, 0, 2)

In [8]:
res_filtered = multipoles.filter_results(results, {'atom': 0, 'k': k, 'p': p, 'r': r, 'nu': (k+p)%2})
res_filtered

,species,atom,nu,l1,l2,k,p,r,t,value
64,1,0,0,2,2,2,0,2,-2,0.020-0.182j
65,1,0,0,2,2,2,0,2,-1,0.000+0.000j
66,1,0,0,2,2,2,0,2,0,-0.018+0.000j
67,1,0,0,2,2,2,0,2,1,0.000+0.000j
68,1,0,0,2,2,2,0,2,2,0.020+0.182j


In [9]:
multipoles.transform_results_to_real(res_filtered)

,species,atom,nu,l1,l2,k,p,r,t,value
64,1,0,0,2,2,2,0,2,-2,0.257
65,1,0,0,2,2,2,0,2,-1,0.000
66,1,0,0,2,2,2,0,2,0,-0.018
67,1,0,0,2,2,2,0,2,1,0.000
68,1,0,0,2,2,2,0,2,2,0.029


In [10]:
with open('BMRO_benchmark/TENSMOM.R1.OUT', 'r') as file:
    res_vasp = multipoles.filter_results(read_from_dft.read_multipoles_from_vasp(file), {'atom': 7, 'l1': 2, 'l2': 2,
                                                                                         'k': k, 'p': p, 'r': r, 'nu': (k+p)%2})

In [11]:
multipoles.transform_results_to_real(res_vasp)

,species,atom,nu,l1,l2,k,p,r,t,value
240,1,7,0,2,2,2,0,2,-2,0.257
241,1,7,0,2,2,2,0,2,-1,0.000
242,1,7,0,2,2,2,0,2,0,-0.018
243,1,7,0,2,2,2,0,2,1,0.000
244,1,7,0,2,2,2,0,2,2,0.029


Indeed, the multipoles from this code are the same as calculated in VASP directly.

# Calculates exchange and Hartree energy

In [12]:
xc_energies = multipoles.calculate_hartree_and_exchange_energies(l, results, uj=(2, 0))

Calculated Slater integrals = (2, 0.0, 0.0)


In [13]:
xc_energies

,species,atom,nu,l1,l2,k,p,r,w.w,exchange F0,exchange F2,exchange F4,hartree F0,hartree F2,hartree F4,exchange total,hartree total
0,1,0,0,2,2,0,0,0,16.210,-0.811,-0.232,-0.232,8.105,0.000,0.000,-1.621,16.210
1,1,0,0,2,2,0,1,1,0.000,-0.000,-0.000,-0.000,0.000,0.000,0.000,0.000,0.000
2,1,0,0,2,2,1,0,1,0.000,-0.000,-0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,1,0,0,2,2,1,1,0,0.024,-0.001,-0.000,0.000,0.000,0.000,0.000,-0.002,0.000
4,1,0,0,2,2,1,1,1,0.027,-0.003,-0.000,0.001,0.000,0.000,0.000,-0.006,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,1,1,1,2,2,3,1,4,0.000,-0.000,0.000,0.000,0.000,0.000,0.000,-0.000,0.000
68,1,1,1,2,2,4,0,4,0.000,-0.000,-0.000,-0.000,0.000,0.000,0.000,-0.000,0.000
69,1,1,1,2,2,4,1,3,0.497,-0.001,-0.000,-0.000,0.000,0.000,0.000,-0.002,0.000
70,1,1,1,2,2,4,1,4,5.460,-0.020,-0.002,-0.000,0.000,0.000,0.000,-0.039,0.000
